<h1>Pricing Analysis</h1>
<p>This project demonstrates several methods that could be used in determining price for a given range of products.</p>
<p>The data used is a list of metals, with made-up figures for price and sales over an arbitrary timeframe.</p>
<p>While we use gold, silver etc, this could be a stand-in for any range of similar products, like washing machines or mattresses. The assumption is that they are different quality versions of similar products.</p>

In [19]:
# import libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

<h1>Constructing the Data</h1>

In [20]:
# tuple of product catalogue with price and sales
stock = {'gold':{'price':100, 'sales':10, 'supply_price': 50},
         'silver':{'price':70,'sales':50, 'supply_price': 30},
         'steel':{'price':50,'sales':80, 'supply_price':25},
         'tamahagane':{'price':45,'sales':90, 'supply_price':20},
         'bronze':{'price':10, 'sales':100, 'supply_price':4},
         'copper':{'price':8,'sales':120, 'supply_price':1}}
stock_df = pd.DataFrame(stock)
stock_df

,gold,silver,steel,tamahagane,bronze,copper
price,100,70,50,45,10,8
sales,10,50,80,90,100,120
supply_price,50,30,25,20,4,1


In [21]:
#create arrays of prices and sales
price = np.array([stock[item]['price'] for item in stock])
sales = np.array([stock[item]['sales'] for item in stock])
supply_prices = np.array([stock[item]['supply_price'] for item in stock])
all_prices = np.array([i for i in range(1,101)])
all_sales = [i for i in range(1,101)]

<h1>Demand Modelling</h1>
<p>In this section we use Linear Regression to try and establish the relationship between price and sales for these products.</p>

<h2>Plotting the Actual Price/Sales Distribution</h2>
Hover over the points to see the product x, y values and name.
Click and drag to select a smaller area. Click the house button at the graph's top right to reset the visual.

In [22]:
fig1 = go.Figure(data=[go.Scatter(x=sales, y=price, text=[item for item in stock], name='Actual')])

fig1.update_layout(
    title='Sales by Price and Quantity',
    xaxis_title='Sale Quantity',
    yaxis_title='Sale Price'
    
)
fig1.show()

<h1>Fit and Test Regression Models</h2>

<h2>Linear Regression</h2>

In [23]:
regr = linear_model.LinearRegression()
regr.fit(price.reshape(-1,1), sales)

LinearRegression()

In [24]:
intercept = regr.intercept_
coef = regr.coef_
sales_pred = regr.predict(price.reshape(-1,1))
r2 = r2_score(sales, sales_pred)
mse = mean_squared_error(sales, sales_pred)
print(f'coef: {coef}, intercept: {intercept}, r-squared: {r2}, mse: {mse}')

coef: [-1.0759781], intercept: 125.75030044064627, r-squared: 0.9322829612463872, mse: 87.46784172341654


<h2>Logarithmic Model</h2>

In [25]:
log_prices = np.log(price)
log_model = linear_model.LinearRegression()
log_model.fit(log_prices.reshape(-1,1), sales)
intercept = log_model.intercept_
coef = log_model.coef_
log_sales_pred = log_model.predict(log_prices.reshape(-1,1))
r2 = r2_score(sales, log_sales_pred)
mse = mean_squared_error(sales, log_sales_pred)
print(f'coef: {coef}, intercept: {intercept}, r-squared: {r2}, mse: {mse}')

coef: [-32.33624779], intercept: 187.93099082106096, r-squared: 0.7416428751382155, mse: 333.71128627980505


<h2>Exponential Model</h2>

In [26]:
log_sales = np.log(sales)
exp_model = linear_model.LinearRegression()
exp_model.fit(price.reshape(-1,1), log_sales)
intercept = exp_model.intercept_
coef = exp_model.coef_
exp_sales_pred = exp_model.predict(price.reshape(-1,1))
r2 = r2_score(sales, exp_sales_pred)
mse = mean_squared_error(sales, exp_sales_pred)
print(f'coef: {coef}, intercept: {intercept}, r-squared: {r2}, mse: {mse}')

coef: [-0.02323808], intercept: 5.177580561031846, r-squared: -3.8539186375683867, mse: 6269.6449068591655


Graphing the models against the actual data. Exponential model omitted due to poor results.

In [27]:
fig1.add_trace(go.Scatter(x=sales_pred, y=price, name='Linear Prediction'))
fig1.add_trace(go.Scatter(x=log_sales_pred, y=price, name='Logarithmic Prediction'))
# fig1.add_trace(go.Scatter(x=exp_sales_pred, y=price, name='Exponential Prediction')) omitted due to poor fit

<p>Given the small dataset we can only really extract rough indicators, but of the linear models tested the standard linear model has the lowest Mean Squared Error and a quite decent R-squared score.
While there are outliers, we should assume a linear relationship between price and sales.</p>
<p>Next we graph the linear model for all sale quantities.</p>
<p>The plot below graphs the predicted sales using the linear model for all (integer) price points. Thus if we are to introduce a new product, we can predict for example, we could sell 60 units at a price of £61, or 101 units at a price of £23.</P>
<p>This obviously does not take account of the profit margins involved, which is for later discussion.</P>

In [28]:
#plot entire linear model
linear_sales_pred = regr.predict(all_prices.reshape(-1,1))
log_sales_pred = log_model.predict(all_prices.reshape(-1,1))
fig2 = go.Figure(data=[
    go.Scatter(x=sales,y=price, name='Actual'),
    go.Scatter(x=linear_sales_pred, y=all_prices, name='Linear Model')
])
fig2.update_layout(
    title='Sales Price Model',
    xaxis_title='Predicted Sale Quantity',
    yaxis_title='Sale Price'
    
)

<h2>Re-Pricing a Product</h2>
We can use the model to estimate the ideal price for a target number of sales using the linear model's intercept and coefficient.

In [29]:
#formula to get price given target sales.
def sale_price(sale):
    return (sale - regr.intercept_) / regr.coef_[0]

In [30]:
#generate price values for all levels of sales
demand_array = [sale_price(sale) for sale in all_sales]

In [31]:
#example of gold - for a sale rate of 10, it appears underpriced.
gold_demand = sale_price(stock['gold']['sales'])
print(gold_demand)

107.57681806899974


<p>Taking the example of gold, it underperforms relative to the model (it sells less than expected for its price). At a price of £100 the expected sale rate is 18. Conversely, at a sale rate of 10, the expected price is £107.
There are many possible reasons:</p>
<ul>
    <li>There's a genuine lack of demand for more gold;</li>
    <li>It's an outlier, outliers happen and no model is perfect;</li>
    <li>It's overpriced and the elasticity of demand is greater than expected at higher price points;</li>
</ul>
<p>
If there is a lack of demand, we may want to try raising the price of gold and see if sales decrease. However, we should consider that demand *should* increase as price decreases, and the profit curve for gold supports a strategy of aiming for more sales at a lower price point.
</p>
<p>
First, let's review our demand model.
</P>
<p>
For each sale quantity, we predict the ideal price. As we've seen with gold, this will not always be totally accurate, but is a rule of thumb based on the best model we've created.
</p>


In [32]:
demand_plot = go.Figure(
    data=[
        go.Scatter(x=all_sales, y=demand_array, name='Demand Curve')
    ]
)
demand_plot.update_layout(
    title='Demand Model',
    xaxis_title='Sale Quantity',
    yaxis_title='Predicted Sale Price'
)
demand_plot.show()

<p>We know that gold is performing worse than expected. This may well just be noise, but could also indicate that there is a genuine lack of demand.</p>
<p>The graph above computes the price expected based on number of sales, rather than sales based on price.</p>
<p>We can calculate for each product the most profitable number of sales. This is x sales for the maximum possible: (expected price to get x sales - supply price) * x sales.</p>
<p>To get a few high margin sales we price the product high, to get many low margin sales we price it low. There is an ideal point between these extremes where the maximum profit is generated.</p>

In [33]:
# calculate the profit as: for each number of sales, the ideal price minus supply cost, the multiply bu number of sales.
gold_unit_profit = [((sale_price(sale) - stock['gold']['supply_price']) * sale) for sale in all_sales]
silver_unit_profit = [((sale_price(sale) - stock['silver']['supply_price']) * sale) for sale in all_sales]
steel_unit_profit = [((sale_price(sale) - stock['steel']['supply_price']) * sale) for sale in all_sales]
tamahagane_unit_profit = [((sale_price(sale) - stock['tamahagane']['supply_price']) * sale) for sale in all_sales]
bronze_unit_profit = [((sale_price(sale) - stock['bronze']['supply_price']) * sale) for sale in all_sales]
copper_unit_profit = [((sale_price(sale) - stock['copper']['supply_price']) * sale) for sale in all_sales]

Display profit curve for each product:

In [34]:
profit_plot = go.Figure(
    data=[
        go.Scatter(x=all_sales,y=gold_unit_profit, name='Gold Profit'),
        go.Scatter(x=all_sales,y=silver_unit_profit, name='Silver Profit'),
        go.Scatter(x=all_sales,y=steel_unit_profit, name='Steel Profit'),
        go.Scatter(x=all_sales,y=tamahagane_unit_profit, name='Tamahagane Profit'),
        go.Scatter(x=all_sales,y=bronze_unit_profit, name='Bronze Profit'),
        go.Scatter(x=all_sales,y=copper_unit_profit, name='Copper Profit')
    ]
)
profit_plot.update_layout(
    title='Profit Curves',
    xaxis_title='Sale Quantity',
    yaxis_title='Profit'
)
profit_plot.show()

<p>The peak of these curves represents the point where the price point is optimised to get the ideal number of sales.</p>
<p>We can now estimate, from the maximum profit, the ideal number of sales and thus the ideal sale price for each product.</p>

In [35]:
profits = {'gold':{'max_profit':round(max(gold_unit_profit),2),'target_sales':gold_unit_profit.index(max(gold_unit_profit)) + 1,'ideal_price': round(sale_price(gold_unit_profit.index(max(gold_unit_profit)) + 1),2)},
           'silver':{'max_profit':round(max(silver_unit_profit),2),'target_sales':silver_unit_profit.index(max(silver_unit_profit)) + 1,'ideal_price': round(sale_price(silver_unit_profit.index(max(silver_unit_profit)) + 1),2)},
           'steel':{'max_profit':round(max(steel_unit_profit),2),'target_sales':steel_unit_profit.index(max(steel_unit_profit)) + 1,'ideal_price': round(sale_price(steel_unit_profit.index(max(steel_unit_profit)) + 1),2)},
           'tamahagane':{'max_profit':round(max(tamahagane_unit_profit),2),'target_sales':tamahagane_unit_profit.index(max(tamahagane_unit_profit)) + 1,'ideal_price': round(sale_price(tamahagane_unit_profit.index(max(tamahagane_unit_profit)) + 1),2)},
           'bronze':{'max_profit':round(max(bronze_unit_profit),2),'target_sales':bronze_unit_profit.index(max(bronze_unit_profit)) + 1,'ideal_price': round(sale_price(bronze_unit_profit.index(max(bronze_unit_profit)) + 1),2)},
           'copper':{'max_profit':round(max(copper_unit_profit),2),'target_sales':copper_unit_profit.index(max(copper_unit_profit)) + 1,'ideal_price': round(sale_price(copper_unit_profit.index(max(copper_unit_profit)) + 1),2)}
           }
profits_df = pd.DataFrame(profits)

<p>For presentation, join our new values back into our original data and calculate some useful columns.</p>

In [36]:
results = pd.concat([stock_df.transpose(), profits_df.transpose()],axis=1)
results['current_profit'] = (results['price'] - results['supply_price']) * results['sales']
results['lost_profit'] = results['max_profit'] - results['current_profit']
results['lost_sales'] = results['target_sales'] - results['sales']
results['price_adjustment'] = results['ideal_price'] - results['price']
results

,price,sales,supply_price,max_profit,target_sales,ideal_price,current_profit,lost_profit,lost_sales,price_adjustment
gold,100,10,50,1202.86,36.0,83.41,500,702.86,26.0,-16.59
silver,70,50,30,2029.91,47.0,73.19,2000,29.91,-3.0,3.19
steel,50,80,25,2270.21,49.0,71.33,2000,270.21,-31.0,21.33
tamahagane,45,90,20,2524.21,52.0,68.54,2250,274.21,-38.0,23.54
bronze,10,100,4,3426.86,61.0,60.18,600,2826.86,-39.0,50.18
copper,8,120,1,3611.42,62.0,59.25,840,2771.42,-58.0,51.25


<p>Referring back to our current actual values, we can see that given our supply costs:</p>
<ul>
<li>Gold should be adjusted down £16.59</li>
<li>Silver is about right but could go up £3.19</li>
<li>Steel should go up £21.33</li>
<li>Tamahagane should go up £23.54</li>
<li>Bronze should go up £50.18</li>
<li>Copper should go up £51.25</li>
</ul>

<h1>Commentary</h1>
<p>In the real world, gold obviously has different market forces operating on it than copper. To reiterate, these products represent a range of similar products, or perhaps the same product at different points in time. We are making an assumption that the demand for these behaves in a similar way.</p>
<p>What is demonstrated here is how linear regression could be applied to a set of prodicts to identify where products are performing above or below what you would typically expect from that set. You EXPECT some products to be atypical.</p>
<p>Bringing some industry knowledge to the table though, these technique can be a starting point and give some direction on how to price a new product or re-price an existing one</p>